In [1]:
import pandas as pd
import numpy as np
import requests
from nba_api.stats import endpoints

In [11]:
#data = endpoints.leagueleaders.LeagueLeaders() 
temporadas = [ '2023-24']
dfs = []

# Iterar sobre cada temporada y obtener las estadísticas de los líderes de la liga
for temporada in temporadas:
    data = endpoints.leagueleaders.LeagueLeaders(season=temporada)
    df = data.league_leaders.get_data_frame()
    # Agregar columna de temporada
    df['Temporada'] = temporada
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo
result = pd.concat(dfs, ignore_index=True)

# Guardar el DataFrame combinado en un archivo CSV
result.to_csv('all_players_stats.csv', index=False)




In [13]:
dict_equipo_jugador = result.groupby('TEAM_ID')['PLAYER_ID'].apply(list).to_dict()

def de_que_equipo_es(valor_buscar,dict_equipo_jugador):
    for equipo, jugadores in dict_equipo_jugador.items():
        if valor_buscar in jugadores:
            return equipo
    else:
        print(f"No se encontró")

In [9]:

from nba_api.stats.endpoints import playergamelog

def get_player_season_per(player_id, season):
    # Obtener los registros de juegos del jugador para la temporada especificada
    games_data = playergamelog.PlayerGameLog(player_id=player_id, season=season)
    games = games_data.get_data_frames()[0]
    print(games)
    
    # Filtrar solo las columnas relevantes para calcular el PER
    relevant_columns = ['Game_ID', 'GAME_DATE', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'MIN']
    games_filtered = games[relevant_columns]
    
    # Calcular el PER (Player Efficiency Rating)
    games_filtered.loc[:, 'PER'] = (games_filtered['PTS'] + games_filtered['REB'] + games_filtered['AST'] +
                                games_filtered['STL'] + games_filtered['BLK'] - games_filtered['TOV'] -
                                games_filtered['PF']) / games_filtered['MIN']
    
    
    return games_filtered


Empezamos a crear un dataframe, con los juegos del jugador promediados

In [10]:
player_id = 201935  # ID de James Harden
season = '2023-24'
player_per_stats = get_player_season_per(player_id, season)
player_per_stats.head(10)

   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0      22023     201935  0022301184  APR 12, 2024  LAC vs. UTA  L   10    1   
1      22023     201935  0022301132  APR 07, 2024  LAC vs. CLE  W   26    6   
2      22023     201935  0022301126  APR 05, 2024  LAC vs. UTA  W   33    4   
3      22023     201935  0022301114  APR 04, 2024  LAC vs. DEN  W   38    6   
4      22023     201935  0022301101  APR 02, 2024    LAC @ SAC  L   31    1   
..       ...        ...         ...           ...          ... ..  ...  ...   
67     22023     201935  0022300024  NOV 14, 2023    LAC @ DEN  L   35    7   
68     22023     201935  0022300179  NOV 12, 2023  LAC vs. MEM  L   29    4   
69     22023     201935  0022300014  NOV 10, 2023    LAC @ DAL  L   29    3   
70     22023     201935  0022300160  NOV 08, 2023    LAC @ BKN  L   36    4   
71     22023     201935  0022300151  NOV 06, 2023    LAC @ NYK  L   31    6   

    FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV

C:\Users\ce200\AppData\Local\Temp\ipykernel_21660\1497799751.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_filtered.loc[:, 'PER'] = (games_filtered['PTS'] + games_filtered['REB'] + games_filtered['AST'] +


,Game_ID,GAME_DATE,PTS,REB,AST,STL,BLK,TOV,PF,MIN,PER
0,0022301184,"APR 12, 2024",4,0,5,0,0,1,0,10,0.800000
1,0022301132,"APR 07, 2024",22,5,5,0,0,2,3,26,1.038462
2,0022301126,"APR 05, 2024",13,10,15,0,0,5,1,33,0.969697
3,0022301114,"APR 04, 2024",20,6,8,1,0,1,0,38,0.894737
4,0022301101,"APR 02, 2024",6,5,8,1,0,5,0,31,0.483871
5,0022301079,"MAR 31, 2024",6,6,10,1,0,0,0,36,0.638889
6,0022301064,"MAR 29, 2024",11,8,4,4,0,4,1,37,0.594595
7,0022301052,"MAR 27, 2024",16,5,14,2,1,1,1,39,0.923077
8,0022301043,"MAR 25, 2024",11,3,7,1,0,1,3,34,0.529412
9,0022301028,"MAR 24, 2024",12,4,14,1,1,2,0,34,0.882353


In [7]:
player_per_stats['GAME_DATE'] = pd.to_datetime(player_per_stats['GAME_DATE'], format='%b %d, %Y')
df_sorted = player_per_stats.sort_values(by='GAME_DATE')

# Restablecer los índices del DataFrame
df_sorted.reset_index(drop=True, inplace=True)

# Imprimir las primeras 5 filas del DataFrame ordenado
df_sorted.head()


,Game_ID,GAME_DATE,PTS,REB,AST,STL,BLK,TOV,PF,MIN,PER
0,0022300151,2023-11-06,17,3,6,1,0,2,2,31,0.741935
1,0022300160,2023-11-08,12,8,5,2,0,5,0,36,0.611111
2,0022300014,2023-11-10,14,5,3,2,0,2,0,29,0.758621
3,0022300179,2023-11-12,11,4,3,1,1,1,0,29,0.655172
4,0022300024,2023-11-14,21,2,4,0,0,1,3,35,0.657143


In [39]:

# Suponiendo que ya tienes el DataFrame df_sorted ordenado por fecha
# Eliminar las primeras 10 filas

# Calcular el promedio del PER de los últimos 10 partidos para cada juego
promedio_per_ultimos_10 = df_sorted['PER'].rolling(window=10, min_periods=1).mean()

# Crear un nuevo DataFrame con los datos requeridos
nuevo_df = pd.DataFrame({
    'Game_ID': df_sorted['Game_ID'],  # Columna Game_ID
    'Promedio_PER_ultimos_10': promedio_per_ultimos_10,  # Columna del promedio del PER de los últimos 10 partidos
    })

# Imprimir las primeras filas del nuevo DataFrame
nuevo_df=nuevo_df.iloc[10:]
nuevo_df['Player_ID']=player_id

In [40]:
nuevo_df

,Game_ID,Promedio_PER_ultimos_10,Player_ID
10,0022300257,0.702758,201935
11,0022300264,0.728826,201935
12,0022300273,0.720531,201935
13,0022300280,0.743586,201935
14,0022301215,0.777871,201935
...,...,...,...
67,0022301101,0.691494,201935
68,0022301114,0.728027,201935
69,0022301126,0.766663,201935
70,0022301132,0.776065,201935


In [ ]:

def de_que_equipo_es(valor_buscar,dict_equipo_jugador):
    for equipo, jugadores in dict_equipo_jugador.items():
        if valor_buscar in jugadores:
            return equipo
    else:
        print(f"No se encontró")

In [26]:
from nba_api.stats.endpoints import commonteamroster, commonallplayers

# Obtener todos los jugadores de la temporada 2023-24
players_data = commonallplayers.CommonAllPlayers(is_only_current_season=1, league_id='00', season='2023-24')

In [36]:
players_df = players_data.get_data_frames()[0]
players_df.to_csv('hola.csv')



In [38]:
df_filtrado = players_df[players_df['ROSTERSTATUS'] == 1]
df_filtrado

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_SLUG,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2023,precious_achiuwa,precious_achiuwa,1610612752,New York,Knicks,NYK,knicks,knicks,Y,00
1,203500,"Adams, Steven",Steven Adams,1,2013,2023,steven_adams,steven_adams,1610612745,Houston,Rockets,HOU,rockets,rockets,Y,00
2,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2023,bam_adebayo,bam_adebayo,1610612748,Miami,Heat,MIA,heat,heat,Y,00
3,1630534,"Agbaji, Ochai",Ochai Agbaji,1,2022,2023,ochai_agbaji,ochai_agbaji,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
4,1630583,"Aldama, Santi",Santi Aldama,1,2021,2023,santi_aldama,santi_aldama,1610612763,Memphis,Grizzlies,MEM,grizzlies,grizzlies,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,201152,"Young, Thaddeus",Thaddeus Young,1,2007,2023,thaddeus_young,thaddeus_young,1610612756,Phoenix,Suns,PHX,suns,suns,Y,00
578,1629027,"Young, Trae",Trae Young,1,2018,2023,trae_young,trae_young,1610612737,Atlanta,Hawks,ATL,hawks,hawks,Y,00
579,1630209,"Yurtseven, Omer",Omer Yurtseven,1,2020,2023,omer_yurtseven,omer_yurtseven,1610612762,Utah,Jazz,UTA,jazz,jazz,Y,01
580,203469,"Zeller, Cody",Cody Zeller,1,2013,2023,cody_zeller,cody_zeller,1610612740,New Orleans,Pelicans,NOP,pelicans,pelicans,Y,00


In [33]:
from nba_api.stats.endpoints import commonteamroster, commonallplayers

# Obtener todos los jugadores de la temporada 2023-24
players_data = commonallplayers.CommonAllPlayers(is_only_current_season=1, league_id='00', season='2023-24')
players_df = players_data.get_data_frames()[0]

# Obtener los IDs de todos los equipos de la NBA
teams_data = commonteamroster.CommonTeamRoster(team_id=1610612752)
teams_df = teams_data.get_data_frames()[0]

# Iterar sobre cada equipo para obtener su roster y fusionar con el DataFrame de jugadores
roster_dfs = []
for team_id in teams_df['TeamID']:
    roster_data = commonteamroster.CommonTeamRoster(team_id=team_id)
    roster_df = roster_data.get_data_frames()[0]
    roster_dfs.append(roster_df)

# Concatenar todos los DataFrames de rosters en uno solo
rosters_df = pd.concat(roster_dfs)

# Fusionar los DataFrames para obtener el TeamID de cada jugador
merged_df = pd.merge(players_df, rosters_df, how='inner', left_on='PERSON_ID', right_on='PLAYER_ID')

# Seleccionar las columnas necesarias
result_df = merged_df[['PERSON_ID', 'TeamID']]

# Renombrar las columnas
result_df.columns = ['PlayerID', 'TeamID']

# Imprimir las primeras filas del DataFrame resultante
print(result_df.head())


   PlayerID      TeamID
0   1630173  1610612752
1   1630173  1610612752
2   1630173  1610612752
3   1630173  1610612752
4   1630173  1610612752
